In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_rest = pd.read_parquet('../Data Engineering/Unification/Final Unifications/df_restaurants.parquet')
df_unificado = pd.read_parquet('../Data Engineering/Unification/Final Unifications/df_reduct_unified.parquet')

In [3]:
def loc_recommend(user_preferences):
    #relacion de categorias
    mapper = {'Cafe':0, 'Family':2, 'American':3, 'European':7, 'Asian':8,
       'Central American':10, 'Vegetarian':6, 'Fastfood':4, 'No Detail':12,
       'South American':11, 'Night':5, 'African':9, 'Breakfast':1}
    
    # generacion de dataframes basicos
    df = df_unificado[(df_unificado.county == user_preferences['county']) & (df_unificado.state == user_preferences['state'])]
    content_features = df[['avg_rating','category']]
    user_preferences = pd.DataFrame(user_preferences,index=[0])
    categories = pd.DataFrame(columns=mapper.keys())

    # mapeo de los dataframes (puede hacerse antes de la funcion)
    def map_category(category):
        return 1 if category == key else 0
    for key in mapper.keys():
        categories[key] = user_preferences['category'].map(map_category)
    for key in mapper.keys():
        content_features[key] = content_features['category'].map(map_category)
    
    # bases finales
    content_features.drop(columns='category',inplace=True)
    user_preferences = pd.merge(user_preferences.drop(columns=['category','state','county']),categories,left_index=True,right_index=True)

    # Matrices de similitud
    similarity_scores = cosine_similarity(content_features, user_preferences).flatten()
    similarity_df = pd.DataFrame({
        'business_id': df['business_id'].values,
        'Cosine': similarity_scores
    })

    # eleccion del top 5
    indices = similarity_df.sort_values(by='Cosine', ascending=False)['business_id'].drop_duplicates().values[:5]

    # toma de datos de los ids seleccionados
    recommended_restaurants = df_rest[df_rest['business_id'].isin(indices)].drop_duplicates(subset='business_id').reset_index(drop=True)
    recommended_restaurants.drop(columns=['%_competition','longitude','latitude','cluster','cluster_rating','cluster_name','review_count'],inplace=True)
    
    return recommended_restaurants
        

In [4]:
user_preferences = {
    'category':'Family',
    'state': 'Florida',
    'county':'Pinellas County',
    'avg_rating': 4.8}

loc_recommend(user_preferences)

,business_id,business_name,category,avg_rating,address,state,city,postal_code,county
0,RGOAfCxhjhc9e5qcu0Bx1A,Wild Cajun Blue Crab & Shrimp,Family,5.0,640 49th St S,Florida,St. Petersburg,33707,Pinellas County
1,0x88c2fd4b6db6ca95:0x5b414c5c84a4c5e0,"Grillin N Chillin St. Pete, Inc.",Family,4.9,"Grillin N Chillin St. Pete, Inc., 6708 Gulf Bl...",Florida,St Pete Beach,33706,Pinellas County
2,0x88c2f16345b985f5:0x77da3436abda1adb,The Black Pearl of Dunedin,Family,4.7,"The Black Pearl of Dunedin, 315 Main St, Duned...",Florida,Dunedin,34698,Pinellas County
3,0x88c2e34cab3e526d:0xc28e84ea2ca45c1f,Fo'Cheezy Twisted Meltz,Family,4.8,"Fo'Cheezy Twisted Meltz, 6305 Gulf Blvd, St Pe...",Florida,St Pete Beach,33706,Pinellas County
4,0x88c2e18601b1a787:0x9b2e8460d54f8477,Mullet's Fish Camp & Market,Family,4.8,"Mullet's Fish Camp & Market, 3901 6th St S, St...",Florida,St. Petersburg,33705,Pinellas County
